In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Kaitangata,NZ,2020-09-06 20:44:07,-46.28,169.85,48.00,72,34,8.99
1,1,Carnarvon,AU,2020-09-06 20:46:38,-24.87,113.63,68.00,94,75,8.05
2,2,Baykit,RU,2020-09-06 20:46:38,61.67,96.37,41.45,82,28,1.45
3,3,Ancud,CL,2020-09-06 20:46:38,-41.87,-73.82,55.99,74,0,1.99
4,4,Tiznit,MA,2020-09-06 20:46:39,29.58,-9.50,83.71,36,0,1.81


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
# Heatmap of percent wind
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 90


In [8]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
4,4,Tiznit,MA,2020-09-06 20:46:39,29.58,-9.50,83.71,36,0,1.81
5,5,Billings,US,2020-09-06 20:46:11,45.78,-108.50,81.00,24,1,6.93
7,7,Kapaa,US,2020-09-06 20:43:26,22.08,-159.32,84.20,66,20,16.11
14,14,Fukuma,JP,2020-09-06 20:46:40,33.77,130.47,89.01,78,75,51.67
30,30,Casian,PH,2020-09-06 20:46:44,11.04,119.71,84.02,70,93,5.99
48,48,Glen Carbon,US,2020-09-06 20:46:47,38.75,-89.98,88.00,70,40,19.46
49,49,Evans,US,2020-09-06 20:46:47,33.53,-82.13,89.60,24,1,9.17
51,51,Westport,US,2020-09-06 20:43:02,41.14,-73.36,81.00,42,75,8.05
52,52,Basco,PH,2020-09-06 20:45:42,20.45,121.97,83.37,65,95,7.78
53,53,Elizabeth City,US,2020-09-06 20:44:44,36.29,-76.25,84.20,45,1,16.11


In [9]:
preferred_cities_df.count()

City_ID       108
City          108
Country       108
Date          108
Lat           108
Lng           108
Max Temp      108
Humidity      108
Cloudiness    108
Wind Speed    108
dtype: int64

In [12]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Tiznit,MA,83.71,29.58,-9.50,
5,Billings,US,81.00,45.78,-108.50,
7,Kapaa,US,84.20,22.08,-159.32,
14,Fukuma,JP,89.01,33.77,130.47,
30,Casian,PH,84.02,11.04,119.71,
48,Glen Carbon,US,88.00,38.75,-89.98,
49,Evans,US,89.60,33.53,-82.13,
51,Westport,US,81.00,41.14,-73.36,
52,Basco,PH,83.37,20.45,121.97,
53,Elizabeth City,US,84.20,36.29,-76.25,


In [13]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

# Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try: 
      hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Tiznit,MA,83.71,29.58,-9.50,
5,Billings,US,81.00,45.78,-108.50,Best Western Plus Clocktower Inn
7,Kapaa,US,84.20,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
14,Fukuma,JP,89.01,33.77,130.47,Masago
30,Casian,PH,84.02,11.04,119.71,
...,...,...,...,...,...,...
551,Urbano Santos,BR,84.20,-3.21,-43.40,Pousada Santa Luzia
552,Esmeralda,CU,88.72,21.85,-78.12,campismo las palmas
556,Akyab,MM,82.60,20.15,92.90,Hotel Memory
557,Lethem,GY,88.45,3.38,-59.80,Savannah Inn


In [14]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
